In [16]:
! pip3 install redis
! pip3 install pymongo
! pip3 install confluent_kafka

    100% |################################| 7.4MB 224kB/s eta 0:00:01


In [109]:
from confluent_kafka import Consumer, KafkaException
import redis
import pymongo
import confluent_kafka
import logging
import sys
import json

In [110]:
def connect_redis():
    
    ORDERBOOK  = 3
    
    return redis.StrictRedis(

        host='127.0.0.1',

        port=6379,

        db=ORDERBOOK)
    

In [111]:
redis_conn = connect_redis()

In [123]:
def connect_mongo():
    myclient = pymongo.MongoClient("mongodb://localhost:27017/", 
                                   username='root',
                                   password='password')

    mydb = myclient["upbit"]
    mycol = mydb["orderbook"]
    
    return mycol

In [124]:
mongo_col = connect_mongo()

In [125]:
def connect_kafka_as_a_consumer():
    # Create logger for consumer (logs will be emitted when poll() is called)
    logger = logging.getLogger('consumer')
    logger.setLevel(logging.DEBUG)
    handler = logging.StreamHandler()
    handler.setFormatter(logging.Formatter('%(asctime)-15s %(levelname)-8s %(message)s'))
    logger.addHandler(handler)
    
    conf = {'bootstrap.servers': "localhost:9092,localhost:9093,localhost:9094",
        'group.id': "orderbook_saver",
        'auto.offset.reset': 'earliest'}
    
    return Consumer(conf, logger=logger)

In [126]:
consumer = connect_kafka_as_a_consumer()

2019-12-05 18:56:08,926 WARNING  MAXPOLL [rdkafka#consumer-7] [thrd:main]: Application maximum poll interval (300000ms) exceeded by 369ms (adjust max.poll.interval.ms for long-running message processing): leaving group
2019-12-05 18:56:08,926 WARNING  MAXPOLL [rdkafka#consumer-7] [thrd:main]: Application maximum poll interval (300000ms) exceeded by 369ms (adjust max.poll.interval.ms for long-running message processing): leaving group
2019-12-05 18:56:08,926 WARNING  MAXPOLL [rdkafka#consumer-7] [thrd:main]: Application maximum poll interval (300000ms) exceeded by 369ms (adjust max.poll.interval.ms for long-running message processing): leaving group
2019-12-05 18:56:08,926 WARNING  MAXPOLL [rdkafka#consumer-7] [thrd:main]: Application maximum poll interval (300000ms) exceeded by 369ms (adjust max.poll.interval.ms for long-running message processing): leaving group
2019-12-05 18:56:08,926 WARNING  MAXPOLL [rdkafka#consumer-7] [thrd:main]: Application maximum poll interval (300000ms) exce

In [127]:
def print_assignment(consumer, partitions):
        print('Assignment:', partitions)

In [128]:
def data_filtering(data):
    del data["stream_type"]
    return data

In [129]:
def extract_key_of_data(data):
    return data["timestamp"]

In [130]:
def save_redis(conn,key,data):
    data = json.dumps(data)
    print ('Set Record:', conn.set(key, data))

In [134]:
def save_mongo(col,data):
    x = col.insert_one(data)
    print(x.inserted_id)

In [132]:
consumer.subscribe(["orderbook"], on_assign=print_assignment)

In [133]:
count = 0

while True:
    try:
        msg = consumer.poll(1.0)
            
        if msg is None:
            continue
        if msg.error():
            raise KafkaException(msg.error())
        else:
            # Proper message
            data = json.loads(msg.value()) 
            data = data_filtering(data)
            key = extract_key_of_data(data)
            save_redis(redis_conn,key,data)
            save_mongo(mongo_col,key,data)
        if count > 10000:
            consumer.close()
            break
        count += 1
    except KeyboardInterrupt:
        sys.stderr.write('%% Aborted by user\n')
        consumer.close()
        break

Assignment: [TopicPartition{topic=orderbook,partition=0,offset=-1001,error=None}]
Set Record: True
5de952d93bedf53133211b43
Set Record: True
5de952d93bedf53133211b44
Set Record: True
5de952d93bedf53133211b45
Set Record: True
5de952d93bedf53133211b46
Set Record: True
5de952d93bedf53133211b47
Set Record: True
5de952d93bedf53133211b48
Set Record: True
5de952d93bedf53133211b49
Set Record: True
5de952d93bedf53133211b4a
Set Record: True
5de952d93bedf53133211b4b
Set Record: True
5de952d93bedf53133211b4c
Set Record: True
5de952d93bedf53133211b4d
Set Record: True
5de952d93bedf53133211b4e
Set Record: True
5de952d93bedf53133211b4f
Set Record: True
5de952da3bedf53133211b50
Set Record: True
5de952da3bedf53133211b51
Set Record: True
5de952da3bedf53133211b52
Set Record: True
5de952da3bedf53133211b53
Set Record: True
5de952da3bedf53133211b54
Set Record: True
5de952da3bedf53133211b55
Set Record: True
5de952da3bedf53133211b56
Set Record: True
5de952da3bedf53133211b57
Set Record: True
5de952da3bedf531332

5de952da3bedf53133211ce4
Set Record: True
5de952da3bedf53133211ce5
Set Record: True
5de952da3bedf53133211ce6
Set Record: True
5de952da3bedf53133211ce7
Set Record: True
5de952da3bedf53133211ce8
Set Record: True
5de952da3bedf53133211ce9
Set Record: True
5de952da3bedf53133211cea
Set Record: True
5de952da3bedf53133211ceb
Set Record: True
5de952da3bedf53133211cec
Set Record: True
5de952da3bedf53133211ced
Set Record: True
5de952da3bedf53133211cee
Set Record: True
5de952da3bedf53133211cef
Set Record: True
5de952da3bedf53133211cf0
Set Record: True
5de952da3bedf53133211cf1
Set Record: True
5de952da3bedf53133211cf2
Set Record: True
5de952da3bedf53133211cf3
Set Record: True
5de952da3bedf53133211cf4
Set Record: True
5de952da3bedf53133211cf5
Set Record: True
5de952da3bedf53133211cf6
Set Record: True
5de952da3bedf53133211cf7
Set Record: True
5de952da3bedf53133211cf8
Set Record: True
5de952da3bedf53133211cf9
Set Record: True
5de952da3bedf53133211cfa
Set Record: True
5de952da3bedf53133211cfb
Set Recor

Set Record: True
5de952da3bedf53133211dda
Set Record: True
5de952da3bedf53133211ddb
Set Record: True
5de952da3bedf53133211ddc
Set Record: True
5de952da3bedf53133211ddd
Set Record: True
5de952da3bedf53133211dde
Set Record: True
5de952da3bedf53133211ddf
Set Record: True
5de952da3bedf53133211de0
Set Record: True
5de952da3bedf53133211de1
Set Record: True
5de952da3bedf53133211de2
Set Record: True
5de952da3bedf53133211de3
Set Record: True
5de952da3bedf53133211de4
Set Record: True
5de952da3bedf53133211de5
Set Record: True
5de952da3bedf53133211de6
Set Record: True
5de952da3bedf53133211de7
Set Record: True
5de952da3bedf53133211de8
Set Record: True
5de952da3bedf53133211de9
Set Record: True
5de952da3bedf53133211dea
Set Record: True
5de952da3bedf53133211deb
Set Record: True
5de952da3bedf53133211dec
Set Record: True
5de952da3bedf53133211ded
Set Record: True
5de952da3bedf53133211dee
Set Record: True
5de952da3bedf53133211def
Set Record: True
5de952da3bedf53133211df0
Set Record: True
5de952da3bedf5313

5de952da3bedf53133211ed2
Set Record: True
5de952da3bedf53133211ed3
Set Record: True
5de952da3bedf53133211ed4
Set Record: True
5de952da3bedf53133211ed5
Set Record: True
5de952da3bedf53133211ed6
Set Record: True
5de952da3bedf53133211ed7
Set Record: True
5de952da3bedf53133211ed8
Set Record: True
5de952da3bedf53133211ed9
Set Record: True
5de952da3bedf53133211eda
Set Record: True
5de952da3bedf53133211edb
Set Record: True
5de952da3bedf53133211edc
Set Record: True
5de952da3bedf53133211edd
Set Record: True
5de952da3bedf53133211ede
Set Record: True
5de952da3bedf53133211edf
Set Record: True
5de952da3bedf53133211ee0
Set Record: True
5de952da3bedf53133211ee1
Set Record: True
5de952da3bedf53133211ee2
Set Record: True
5de952da3bedf53133211ee3
Set Record: True
5de952da3bedf53133211ee4
Set Record: True
5de952da3bedf53133211ee5
Set Record: True
5de952da3bedf53133211ee6
Set Record: True
5de952da3bedf53133211ee7
Set Record: True
5de952da3bedf53133211ee8
Set Record: True
5de952da3bedf53133211ee9
Set Recor

5de952da3bedf53133211fb9
Set Record: True
5de952da3bedf53133211fba
Set Record: True
5de952da3bedf53133211fbb
Set Record: True
5de952da3bedf53133211fbc
Set Record: True
5de952da3bedf53133211fbd
Set Record: True
5de952da3bedf53133211fbe
Set Record: True
5de952da3bedf53133211fbf
Set Record: True
5de952da3bedf53133211fc0
Set Record: True
5de952da3bedf53133211fc1
Set Record: True
5de952da3bedf53133211fc2
Set Record: True
5de952da3bedf53133211fc3
Set Record: True
5de952da3bedf53133211fc4
Set Record: True
5de952da3bedf53133211fc5
Set Record: True
5de952da3bedf53133211fc6
Set Record: True
5de952da3bedf53133211fc7
Set Record: True
5de952da3bedf53133211fc8
Set Record: True
5de952da3bedf53133211fc9
Set Record: True
5de952da3bedf53133211fca
Set Record: True
5de952da3bedf53133211fcb
Set Record: True
5de952da3bedf53133211fcc
Set Record: True
5de952da3bedf53133211fcd
Set Record: True
5de952da3bedf53133211fce
Set Record: True
5de952da3bedf53133211fcf
Set Record: True
5de952da3bedf53133211fd0
Set Recor

...

5de952e13bedf53133213e43
Set Record: True
5de952e13bedf53133213e44
Set Record: True
5de952e13bedf53133213e45
Set Record: True
5de952e13bedf53133213e46
Set Record: True
5de952e13bedf53133213e47
Set Record: True
5de952e13bedf53133213e48
Set Record: True
5de952e13bedf53133213e49
Set Record: True
5de952e13bedf53133213e4a
Set Record: True
5de952e13bedf53133213e4b
Set Record: True
5de952e13bedf53133213e4c
Set Record: True
5de952e13bedf53133213e4d
Set Record: True
5de952e13bedf53133213e4e
Set Record: True
5de952e13bedf53133213e4f
Set Record: True
5de952e13bedf53133213e50
Set Record: True
5de952e13bedf53133213e51
Set Record: True
5de952e13bedf53133213e52
Set Record: True
5de952e13bedf53133213e53
Set Record: True
5de952e13bedf53133213e54
Set Record: True
5de952e13bedf53133213e55
Set Record: True
5de952e13bedf53133213e56
Set Record: True
5de952e13bedf53133213e57
Set Record: True
5de952e13bedf53133213e58
Set Record: True
5de952e13bedf53133213e59
Set Record: True
5de952e13bedf53133213e5a
Set Recor

5de952e13bedf53133213f36
Set Record: True
5de952e13bedf53133213f37
Set Record: True
5de952e13bedf53133213f38
Set Record: True
5de952e13bedf53133213f39
Set Record: True
5de952e13bedf53133213f3a
Set Record: True
5de952e13bedf53133213f3b
Set Record: True
5de952e13bedf53133213f3c
Set Record: True
5de952e13bedf53133213f3d
Set Record: True
5de952e13bedf53133213f3e
Set Record: True
5de952e13bedf53133213f3f
Set Record: True
5de952e13bedf53133213f40
Set Record: True
5de952e13bedf53133213f41
Set Record: True
5de952e13bedf53133213f42
Set Record: True
5de952e13bedf53133213f43
Set Record: True
5de952e13bedf53133213f44
Set Record: True
5de952e13bedf53133213f45
Set Record: True
5de952e13bedf53133213f46
Set Record: True
5de952e13bedf53133213f47
Set Record: True
5de952e13bedf53133213f48
Set Record: True
5de952e13bedf53133213f49
Set Record: True
5de952e13bedf53133213f4a
Set Record: True
5de952e13bedf53133213f4b
Set Record: True
5de952e13bedf53133213f4c
Set Record: True
5de952e13bedf53133213f4d
Set Recor

5de952e13bedf53133214008
Set Record: True
5de952e13bedf53133214009
Set Record: True
5de952e13bedf5313321400a
Set Record: True
5de952e13bedf5313321400b
Set Record: True
5de952e13bedf5313321400c
Set Record: True
5de952e13bedf5313321400d
Set Record: True
5de952e13bedf5313321400e
Set Record: True
5de952e13bedf5313321400f
Set Record: True
5de952e13bedf53133214010
Set Record: True
5de952e13bedf53133214011
Set Record: True
5de952e13bedf53133214012
Set Record: True
5de952e13bedf53133214013
Set Record: True
5de952e13bedf53133214014
Set Record: True
5de952e13bedf53133214015
Set Record: True
5de952e13bedf53133214016
Set Record: True
5de952e13bedf53133214017
Set Record: True
5de952e13bedf53133214018
Set Record: True
5de952e13bedf53133214019
Set Record: True
5de952e13bedf5313321401a
Set Record: True
5de952e13bedf5313321401b
Set Record: True
5de952e13bedf5313321401c
Set Record: True
5de952e13bedf5313321401d
Set Record: True
5de952e13bedf5313321401e
Set Record: True
5de952e13bedf5313321401f
Set Recor

Set Record: True
5de952e23bedf531332140e0
Set Record: True
5de952e23bedf531332140e1
Set Record: True
5de952e23bedf531332140e2
Set Record: True
5de952e23bedf531332140e3
Set Record: True
5de952e23bedf531332140e4
Set Record: True
5de952e23bedf531332140e5
Set Record: True
5de952e23bedf531332140e6
Set Record: True
5de952e23bedf531332140e7
Set Record: True
5de952e23bedf531332140e8
Set Record: True
5de952e23bedf531332140e9
Set Record: True
5de952e23bedf531332140ea
Set Record: True
5de952e23bedf531332140eb
Set Record: True
5de952e23bedf531332140ec
Set Record: True
5de952e23bedf531332140ed
Set Record: True
5de952e23bedf531332140ee
Set Record: True
5de952e23bedf531332140ef
Set Record: True
5de952e23bedf531332140f0
Set Record: True
5de952e23bedf531332140f1
Set Record: True
5de952e23bedf531332140f2
Set Record: True
5de952e23bedf531332140f3
Set Record: True
5de952e23bedf531332140f4
Set Record: True
5de952e23bedf531332140f5
Set Record: True
5de952e23bedf531332140f6
Set Record: True
5de952e23bedf5313

Set Record: True
5de952e23bedf531332141b5
Set Record: True
5de952e23bedf531332141b6
Set Record: True
5de952e23bedf531332141b7
Set Record: True
5de952e23bedf531332141b8
Set Record: True
5de952e23bedf531332141b9
Set Record: True
5de952e23bedf531332141ba
Set Record: True
5de952e23bedf531332141bb
Set Record: True
5de952e23bedf531332141bc
Set Record: True
5de952e23bedf531332141bd
Set Record: True
5de952e23bedf531332141be
Set Record: True
5de952e23bedf531332141bf
Set Record: True
5de952e23bedf531332141c0
Set Record: True
5de952e23bedf531332141c1
Set Record: True
5de952e23bedf531332141c2
Set Record: True
5de952e23bedf531332141c3
Set Record: True
5de952e23bedf531332141c4
Set Record: True
5de952e23bedf531332141c5
Set Record: True
5de952e23bedf531332141c6
Set Record: True
5de952e23bedf531332141c7
Set Record: True
5de952e23bedf531332141c8
Set Record: True
5de952e23bedf531332141c9
Set Record: True
5de952e23bedf531332141ca
Set Record: True
5de952e23bedf531332141cb
Set Record: True
5de952e23bedf5313